In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from crawto.charts.charts import Plot,ScatterChart,BarChart,LineChart

In [3]:
%load_ext blackcellmagic

In [4]:
x=[-1,0,1]
y = [1,-1,1]

l = LineChart(width="eight")
l.add_DataSet("t",x,y)
l.add_DataSet("t1" ,x,[2,2,2])
l.edit_title("LineChart")
l.edit_xAxes("x Axis")
l.edit_yAxes("y Axis","linear")

c = ScatterChart(width="eight")
c.add_DataSet("test",x,y)
c.add_DataSet("t1" ,x,[2,2,2],["a","b","c"])
c.edit_title("ScatterChart")

b = BarChart(width="eight")
b.add_DataSet("t",x,y)
b.add_DataSet("t1" ,x,[2,2,2])
b.edit_title("BarChart")

In [5]:
p = Plot()

In [6]:
p.add_column(c)
p.add_column(b)
p.add_column(l)

In [7]:
p.display